# Making a cubic camembert

The moonshot: I have designed a camembert contact map. Let's see if it actually works!

In [74]:
import numpy as np
from json_dump import *
import contact_utils as cu
import config as cfg
from pathlib import Path
from geometry.cubic import CubicGeometry
from plotting.plot_cubic import plot_cubes_from_simulation_results

In [75]:
### Key parameter: how we will name this run
run_name = "01_camembert_test"

### Define model parameters

model_params = {}

# ---------- LATTICE OPTIONS ----------

# Options:
# "chain", "square", "triangular", "cubic", "bcc", "fcc"
model_params["lattice_name"] = "cubic"

# Lattice dimensions
model_params["lx"] = 18
model_params["ly"] = 18  # Has to be 1 for chain
model_params["lz"] = 18  # Has to be 1 for square & triangular

In [76]:
# ---------- MODEL PARAMETERS ----------

# Number of particle types
model_params["n_types"] = 1

# Number of particles of each type
model_params["n_particles"] = [200]

# Couplings is its own beast. Should be gotten with the appropriate helper
# function.

## Important note: for 3D particles, do not use the get_matrix and set_contact utils!

A lot of face pairs are equivalent under rotation for cubes (e.g. (0, 0), (1, 3), (2, 2), (3, 1)).
When using a full contact matrix and feeding it to the wrapper using `set_single_species_contacts`, if all the equivalent pairs don't
have the same matrix coefficient, some of them will be overwritten by other values, and the contacts will be wrong.

Better to do things the way they are done below

In [86]:
crystal_energy = -18.7
# defect_energy = -10
defect_energy = crystal_energy / 0.8
repel_energy = 10000
# crystal_energy = repel_energy

get_opposite_face = CubicGeometry().get_opposite_face
# First, create a cmap_wrapper object with the appropriate number of particle types
cmap_wrapper = cu.ContactMapWrapper.cubic(
    n_types = model_params["n_types"], init_energy = repel_energy)

# Setting crystalline interactions
for i in range(3):
    face = 4 * i
    opposite_face = get_opposite_face(face)
    cmap_wrapper[face, opposite_face] = crystal_energy

# Setting camembert interactions
# camembert_contacts = [
#     (0, 5),
#     (0, 11),
#     (0, 19),
#     (0, 23),
#     (4, 8),
#     (4, 15),
#     (4, 20),
#     (8, 15),
#     (8, 18),
#     # (12, 19),
#     # (12, 21),
#     # (16, 22),
# ]
camembert_contacts = [
    (12, 17),
    (8, 16),
    (0, 17),
    (16,20),
    # (0, 21),
    (12, 23),
    # (4, 22),
    (8, 13),
    (4, 13),
    (4, 10),
    (0, 7),
    (4,22),
    (0, 21),
    (0, 9),
    (0, 17)
]

# camembert_contacts = [
#     (12, 17),
#     (8, 16),
#     (0, 17),
#     (16,20),
#     # (0, 21),
#     (12, 23),
#     # (4, 22),
#     (8, 13),
#     (4, 13)
# ]
for contact in camembert_contacts:
    cmap_wrapper[contact] = defect_energy

# And process the couplings into flattened form
model_params["couplings"] = cmap_wrapper.get_formatted_couplings()

In [87]:
# Initialization option
model_params["initialize_option"] = "random"

# If "initialize_option" is set to "from_file", we must specify the location of
# the input file
# model_params["state_input"] = str(cfg.structures_path/"final_structure.dat")

# Options for average collection

model_params["state_av_option"] = True
model_params["e_av_option"] = True

if model_params["state_av_option"]:
    state_path = Path("./data/"+run_name+"/average_state")
    state_path.mkdir(parents = True, exist_ok = True)
    model_params["state_av_output"] =  str(state_path.resolve()) + "/"

if model_params["e_av_option"]:
    energy_path = Path("./data/"+run_name+"/average_energy")
    energy_path.mkdir(parents = True, exist_ok = True)
    model_params["e_av_output"] = str(energy_path.resolve())  + "/"

In [88]:
# Pick the probabilities of different moves. Has to sum to 1.
# Options:
""""
    "swap_empty_full",
    "swap_full_full",
    "rotate",
    "mutate",
    "rotate_and_swap_w_empty"
"""

moves_dict = {}
moves_dict["swap_empty_full"] = 1/3
moves_dict["rotate"] = 1/3
moves_dict["rotate_and_swap_w_empty"] = 1/3

model_params["move_probas"] = moves_dict

make_json_file(model_params, cfg.input_path/"model_params.json")

In [89]:
### Define mc parameters

mc_params = {}

# Number of MC steps used for equilibration
mc_params["mcs_eq"] = 10000

# Number of MC steps used for averaging
mc_params["mcs_av"] = 10

# Type of cooling schedule
# if exponential chosen: specify log10(T) as initial and final temperatures
mc_params["cooling_schedule"] = "exponential"

# Initial annealing temperature
mc_params["Ti"] = 2

# Final annealing temperature
mc_params["Tf"] = 0

# Number of annealing steps
mc_params["Nt"] = 40

# Option to collect state checkpoints at the end of each temperature cycle
mc_params["checkpoint_option"] = True

# If checkpoint is True, we need to provide the output address for the 
# checkpoint files
structures_path = Path("./data/"+run_name+"/structures")
structures_path.mkdir(parents = True, exist_ok = True)

if mc_params["checkpoint_option"]:
    mc_params["checkpoint_address"] = str(structures_path.resolve())+"/"

# Output location of the final state configuration (must end with "/")
mc_params["final_structure_address"] = str(structures_path.resolve())+"/"

make_json_file(mc_params, cfg.input_path/"mc_params.json")

In [ ]:
# Run the simulation from inside the notebook.
# Note that the program only writes to the cell output after it is done running.
# If you want to follow the excecution in real time, run the program from the command line!

# Parameter in run_sim is a boolean flag, which will overwrite the simulation results if set to True

cfg.run_simulation(True)

At least one of the output folders is already populated!
overwrite flag set to True: running anyway.


In [85]:
# Plotting module creates a blender file which you have to open yourself
from plotting.plot_cubic import deleteAllObjects

blend_file_path = (f"3dFigures/02_camembert_full_contacts_crystal_{crystal_energy:.02f}"
                   f"_rep_{repel_energy:.02f}_defect_{defect_energy:.02f}.blend")
# blend_file_path = (f"3dFigures/02_camembert_half_contacts_crystal_{crystal_energy:.02f}"
#                    f"_rep_{repel_energy:.02f}_defect_{defect_energy:.02f}.blend")
plot_cubes_from_simulation_results(struct_folder = str(structures_path.resolve()), fig_file = blend_file_path)

200


/var/folders/v3/gbqhvbxs6nz1258v43493tx40000gp/T/ipykernel_70733/59462908.py:6: UserWarning: Gimbal lock detected. Setting third angle to zero since it is not possible to uniquely determine all angles.
  plot_cubes_from_simulation_results(struct_folder = str(structures_path.resolve()), fig_file = blend_file_path)
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.pn

of 'one_cube_numbered.obj' took 0.30 ms
OBJ import of 'one_cube_numbered.obj' took 0.32 ms
OBJ import of 'one_cube_numbered.obj' took 0.26 ms
OBJ import of 'one_cube_numbered.obj' took 0.79 ms
OBJ import of 'one_cube_numbered.obj' took 0.70 ms
OBJ import of 'one_cube_numbered.obj' took 0.66 ms
OBJ import of 'one_cube_numbered.obj' took 0.65 ms
OBJ import of 'one_cube_numbered.obj' took 0.66 ms
OBJ import of 'one_cube_numbered.obj' took 0.70 ms
OBJ import of 'one_cube_numbered.obj' took 0.76 ms
OBJ import of 'one_cube_numbered.obj' took 0.78 ms
OBJ import of 'one_cube_numbered.obj' took 0.48 ms
OBJ import of 'one_cube_numbered.obj' took 0.46 ms
OBJ import of 'one_cube_numbered.obj' took 0.39 ms
OBJ import of 'one_cube_numbered.obj' took 0.36 ms
OBJ import of 'one_cube_numbered.obj' took 0.44 ms
OBJ import of 'one_cube_numbered.obj' took 0.55 ms
OBJ import of 'one_cube_numbered.obj' took 0.45 ms
OBJ import of 'one_cube_numbered.obj' took 0.41 ms
OBJ import of 'one_cube_numbered.obj' took

Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/

Info: Saved "02_camembert_half_contacts_crystal_-18.70_rep_10000.00_defect_-46.75.blend"


Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/oneCube/full_cube_numbered.png'
Loaded image from: '/Users/vincent/research/projects/23_frustratedSelfAssembly/simulations/2404_frusa_lattice_mc/python/src/plotting/assets/

In [5]:
results = cfg.load_structure(struct_folder = structures_path)

NameError: name 'structures_path' is not defined